In [18]:
#import packages
import requests as req
import datetime
import json
import pandas as pd
import os
import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')

import numpy as np                               # vectors and matrices
import pandas as pd                              # tables and data manipulations
import matplotlib.pyplot as plt                  # plots
import seaborn as sns                            # more plots

from dateutil.relativedelta import relativedelta # working with dates with style
from scipy.optimize import minimize              # for function minimization

import statsmodels.formula.api as smf            # statistics and econometrics
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from itertools import product                    # some useful functions
from tqdm import tqdm_notebook

%matplotlib inline



In [19]:
mh_removed = [5, 13, 19, 40, 44, 16, 29]
br_removed = [1, 21, 25, 40, 47, 41]
mh_num = [i for i in range(1,48)]
br_num = [i for i in range(1,53)]
mh_meters = ['MH'+str(item).zfill(2) for item in mh_num]
br_meters = ['BR'+str(item).zfill(2) for item in br_num]
for i in mh_removed:
    mh_meters.remove('MH'+str(i).zfill(2))
for i in br_removed:
    br_meters.remove('BR'+str(i).zfill(2))
meter_list = [mh_meters,br_meters] 
final_meter_list = [item for sublist in meter_list for item in sublist]

In [20]:
# know the number of meters

len(final_meter_list)

86

In [21]:
files = os.listdir('data preprocess/data2')
df_list = []
for f in files:
    df = pd.read_csv('data preprocess/data2/'+f)
    df_list.append(df.iloc[::-1])
df =pd.concat([df_list[i] for i in range(len(df_list))],axis=0)

In [22]:
# preprocess
df['x_Timestamp'] = pd.to_datetime(df['x_Timestamp'])
df.set_index('x_Timestamp',inplace=True)

In [23]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [24]:
#Modify the data

# mh04 and mh38
# drop mh04

df = df.loc[df['meter']!='MH04']

#drop mh32

df = df.loc[df['meter']!='MH32']


In [25]:
df = df.loc[~((df['meter']=='MH25')&(df.index<'2020-06-30'))]

In [26]:
df

,t_kWh,t_kVAh,z_Avg Voltage (Volt),z_Max Voltage (Volt),z_Min Voltage (Volt),z_Avg Current (Amp),z_Max Current (Amp),z_Min Current (Amp),y_Freq (Hz),y_PF Total,meter
x_Timestamp,,,,,,,,,,,
2019-07-09 00:00:00,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,BR02
2019-07-09 00:03:00,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,BR02
2019-07-09 00:06:00,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,BR02
2019-07-09 00:09:00,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,BR02
2019-07-09 00:12:00,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,BR02
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24 23:45:00,0.000,0.000,264.74,265.26,264.28,0.03,0.03,0.03,49.93,-0.46,MH47
2020-12-24 23:48:00,0.000,0.001,265.35,265.62,264.93,0.03,0.03,0.03,49.96,-0.67,MH47
2020-12-24 23:51:00,0.001,0.000,264.91,265.08,264.57,0.03,0.03,0.03,50.00,-0.27,MH47


In [27]:
meter_list = list(df['meter'].unique())

In [28]:
# clean the data
def string_2_date(string):
    return datetime.datetime.strptime(string, '%Y-%m-%d')


# keep a dictionary to record the number of days to be removed meterwise

# function to find out the length of longest sequence of continuos 0's in t_kWh series. 
def max0(sr):
     return (sr != 0).cumsum().value_counts().max() - (0 if (sr != 0).cumsum().value_counts().idxmax()==0 else 1)


# group by date and check if the data is corrupted

def clean_data(df):

    end_date = '2020-12-28'
    
    removed_num = {k:0 for k in meter_list}
    total_num = {k:0 for k in meter_list}
    num_days = {k:0 for k in meter_list}

    keep =[] #If keep==1; keep the row, else remove it.

    
    grouped = df.groupby(['meter',df.index.date]) # Grouping by meter and date

    for name,group in grouped:
        sub = grouped.get_group(name)
        length = sub.shape[0]   # number of timestamps in the day
        zero = max0(sub['z_Avg Voltage (Volt)']) # longest sequence of 0's

        if ((sub['t_kWh']>0.3).any()) or (zero>160):
            keep.append([0]*length)
            removed_num[name[0]] += 1 # update the number of days to be removed in the dictionary
        else:
            keep.append([1]*length) 
        total_num[name[0]] += 1  
        start_date = sub.iloc[0].name.date()
        start_date = start_date.strftime("%Y-%m-%d")
        
        
        
        num_days[name[0]] = string_2_date(start_date)
    return keep,removed_num,total_num,num_days

In [29]:
keep, removed_dict,total_dict,num_days_dict = clean_data(df)
keep_list = [item for sublist in keep for item in sublist]

In [30]:
df['keep'] = keep_list
remove = df.loc[df['keep']==0]
df_new =df.loc[df['keep']==1] #Remove the corrupted data

In [31]:
#df meterwise start_date of receiving data
start = {k:0 for k in meter_list}
grouped = df.groupby('meter')
for name,group in grouped:
    sub = grouped.get_group(name)
    start_date = sub.iloc[0].name.date()
    start_date = start_date.strftime("%Y-%m-%d")
    start[name] = start_date

In [32]:
start

{'BR02': '2019-07-09',
 'BR03': '2019-07-09',
 'BR04': '2019-07-10',
 'BR05': '2019-07-10',
 'BR06': '2019-07-10',
 'BR07': '2019-07-10',
 'BR08': '2019-07-10',
 'BR09': '2019-09-22',
 'BR10': '2019-07-10',
 'BR11': '2019-07-16',
 'BR12': '2019-07-16',
 'BR13': '2019-07-16',
 'BR14': '2019-07-16',
 'BR15': '2019-07-16',
 'BR16': '2019-07-16',
 'BR17': '2019-07-16',
 'BR18': '2019-07-16',
 'BR19': '2019-07-16',
 'BR20': '2019-07-16',
 'BR22': '2019-09-11',
 'BR23': '2019-07-17',
 'BR24': '2019-07-16',
 'BR26': '2019-07-16',
 'BR27': '2019-07-16',
 'BR28': '2019-07-16',
 'BR29': '2019-07-16',
 'BR30': '2019-09-13',
 'BR31': '2019-07-16',
 'BR32': '2019-08-08',
 'BR33': '2019-08-08',
 'BR34': '2019-05-30',
 'BR35': '2019-08-08',
 'BR36': '2019-08-27',
 'BR37': '2019-08-08',
 'BR38': '2019-08-08',
 'BR39': '2019-08-08',
 'BR42': '2019-08-20',
 'BR43': '2019-08-21',
 'BR44': '2019-08-20',
 'BR45': '2019-08-20',
 'BR46': '2019-08-20',
 'BR48': '2019-08-20',
 'BR49': '2019-09-08',
 'BR50': '2

In [34]:
df_meta = pd.DataFrame([removed_dict,total_dict,start])

In [35]:
df_meta= df_meta.T

In [36]:
df_meta['end_date'] = '2020-12-28'

In [37]:
days = {k:0 for k in meter_list}
for ind, row in df_meta.iterrows():
    num = (string_2_date(row['end_date'])-string_2_date(row[2])).days
    days[ind] = num

In [38]:
days

{'BR02': 538,
 'BR03': 538,
 'BR04': 537,
 'BR05': 537,
 'BR06': 537,
 'BR07': 537,
 'BR08': 537,
 'BR09': 463,
 'BR10': 537,
 'BR11': 531,
 'BR12': 531,
 'BR13': 531,
 'BR14': 531,
 'BR15': 531,
 'BR16': 531,
 'BR17': 531,
 'BR18': 531,
 'BR19': 531,
 'BR20': 531,
 'BR22': 474,
 'BR23': 530,
 'BR24': 531,
 'BR26': 531,
 'BR27': 531,
 'BR28': 531,
 'BR29': 531,
 'BR30': 472,
 'BR31': 531,
 'BR32': 508,
 'BR33': 508,
 'BR34': 578,
 'BR35': 508,
 'BR36': 489,
 'BR37': 508,
 'BR38': 508,
 'BR39': 508,
 'BR42': 496,
 'BR43': 495,
 'BR44': 496,
 'BR45': 496,
 'BR46': 496,
 'BR48': 496,
 'BR49': 477,
 'BR50': 462,
 'BR51': 469,
 'BR52': 600,
 'MH01': 607,
 'MH02': 607,
 'MH03': 597,
 'MH06': 607,
 'MH07': 607,
 'MH08': 607,
 'MH09': 607,
 'MH10': 607,
 'MH11': 607,
 'MH12': 607,
 'MH14': 607,
 'MH15': 607,
 'MH17': 607,
 'MH18': 607,
 'MH20': 607,
 'MH21': 607,
 'MH22': 607,
 'MH23': 600,
 'MH24': 600,
 'MH25': 181,
 'MH26': 598,
 'MH27': 598,
 'MH28': 598,
 'MH30': 598,
 'MH31': 597,
 'MH33

In [39]:
df_meta['total_days'] = days.values()

In [40]:
df_meta['missing'] = (1 - (df_meta[1]/df_meta['total_days']))*100

In [41]:
df_meta.to_csv('Cleaning MetaData_corrected_02Mar21.csv')

In [2]:
import pandas as pd

In [33]:
df_new.to_csv('Cleaned data 02Mar21_corrected.csv')
# df_new = pd.read_csv('Cleaned data 02Mar21_malacious.csv')
# df_new = pd.read_csv('Cleaned data 09Feb21.csv')

In [10]:
df_new.index= pd.to_datetime(df_new['x_Timestamp'])

In [42]:
# electricity consumption

grouped = df_new.groupby(['meter',df_new.index.year,df_new.index.month])

In [43]:
monthly = []
meter = []
m_y = []
days = []
v = []
pow_c = []
for name,group in grouped:
    d = []
    p = []
    sub = grouped.get_group(name)
    meter.append(name[0])
    gr = sub.groupby(sub.index.date)
    for n,g in gr:
        s = gr.get_group(n)
        daily = (s['t_kWh'].sum()/s.shape[0])*480
        powec = s.loc[s['z_Avg Voltage (Volt)']==0]
        power_cut = (powec.shape[0]*3)/60
        d.append(daily)
        p.append(power_cut)
    mon = (sum(d)/len(d))*30
    days.append(len(d))
    monthly.append(mon)
    mon_year = str(name[2])+'-'+str(name[1])
    m_y.append(mon_year)
    sub = s.loc[s['z_Avg Voltage (Volt)']!=0]
    v.append(sub['z_Avg Voltage (Volt)'].mean())
    pow_c.append(sum(p)/len(p))
    

In [13]:
df_new

,x_Timestamp,t_kWh,t_kVAh,z_Avg Voltage (Volt),z_Max Voltage (Volt),z_Min Voltage (Volt),z_Avg Current (Amp),z_Max Current (Amp),z_Min Current (Amp),y_Freq (Hz),y_PF Total,meter,keep
x_Timestamp,,,,,,,,,,,,,
2019-07-10 00:00:00,2019-07-10 00:00:00,0.021,0.021,243.10,243.35,242.91,1.79,1.80,1.78,50.02,0.99,BR02,1
2019-07-10 00:03:00,2019-07-10 00:03:00,0.021,0.021,242.91,243.20,242.44,1.80,1.89,1.79,50.07,0.99,BR02,1
2019-07-10 00:06:00,2019-07-10 00:06:00,0.021,0.021,242.46,242.73,242.15,1.83,1.84,1.79,50.00,0.99,BR02,1
2019-07-10 00:09:00,2019-07-10 00:09:00,0.020,0.021,241.27,242.05,240.71,1.79,1.83,1.78,49.95,0.99,BR02,1
2019-07-10 00:12:00,2019-07-10 00:12:00,0.020,0.020,240.77,241.29,238.98,1.79,1.80,1.78,49.98,0.99,BR02,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24 23:45:00,2020-12-24 23:45:00,0.000,0.000,264.74,265.26,264.28,0.03,0.03,0.03,49.93,-0.46,MH47,1
2020-12-24 23:48:00,2020-12-24 23:48:00,0.000,0.001,265.35,265.62,264.93,0.03,0.03,0.03,49.96,-0.67,MH47,1
2020-12-24 23:51:00,2020-12-24 23:51:00,0.001,0.000,264.91,265.08,264.57,0.03,0.03,0.03,50.00,-0.27,MH47,1


In [44]:
df_ec = pd.DataFrame({'Meter':meter,'M-Y':m_y,'num_days_data_available':days,'Consumption':monthly,'Avg Voltage':v, 'Daily Avg Power cut':pow_c})

In [45]:
df_ec.to_csv('Monthly EC 02Mar21_withpowercut.csv')

In [85]:
months = [str(n)+'-2020' for n in range(3,11)]

In [105]:
annual = [str(n)+'-2020' for n in range(1,13)]

In [100]:
m_ec = dict()
month = []
grouped = df_ec.groupby('Meter')
for name,group in grouped:
    sub = grouped.get_group(name)
    s = sub.loc[sub['M-Y'].isin(months)]
    m_ec[name] = s['Consumption'].sum()
    month.append(s.shape[0])

In [101]:
m_ec_df = pd.DataFrame(m_ec,index=[0])

In [102]:
m_ec_df = m_ec_df.T

In [103]:
m_ec_df['num_months_data_available out of Mar-Oct (8)'] = month

In [108]:
m_ec_df['Total consumption (Mar-Oct)'] = (m_ec_df[0]/m_ec_df['num_months_data_available out of Mar-Oct (8)'])*8

In [110]:
m_ec_df.to_csv('total mar-oct consumption 09feb21.csv')

In [111]:
m_ec_df

,0,num_months_data_available out of Mar-Oct (8),Total consumption (Mar-Oct)
BR02,1762.644830,8,1762.644830
BR03,1429.734971,7,1633.982824
BR04,2929.934599,8,2929.934599
BR05,797.975155,8,797.975155
BR06,3586.955389,8,3586.955389
...,...,...,...
MH42,1747.040782,8,1747.040782
MH43,3744.790176,8,3744.790176
MH45,3156.671005,8,3156.671005
MH46,1201.024975,8,1201.024975


In [48]:
df_new = df_new.loc[df_new['meter']=='BR44']

In [50]:
df_new.to_csv('BR44_SM_Data.csv')